In [1]:
import os

In [2]:
%pwd

'c:\\Users\\HP\\Desktop\\Python\\Vs_Python\\Projects\\Kidney_Disease_MLop_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HP\\Desktop\\Python\\Vs_Python\\Projects\\Kidney_Disease_MLop_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):# Assigning config path in variable

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)# Auto access the config path

        create_directories([self.config.artifacts_root])# Create the artifacts dirs


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion# Access the config data ingestion path & assign in config variable

        create_directories([config.root_dir])# Create the data ingestion dirs

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )# Assign the all the path of data ingestion into a variable

        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url = self.config.source_URL # access the url path
            zip_download_dir = self.config.local_data_file# Access the zip file path
            os.makedirs("artifacts/data_ingestion", exist_ok=True)# ansure the dirs are exist or not
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]# accessing the file id
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)# connecting prefix & file id to dowload the file

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir # access the unzip path
        os.makedirs(unzip_path, exist_ok=True)# ansure the dirs are exist or not
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:# open the zipfile in read mode
            zip_ref.extractall(unzip_path)# extract the zip file

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-15 17:11:55,120: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-15 17:11:55,120: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-15 17:11:55,129: INFO: common: created directory at: artifacts]
[2025-07-15 17:11:55,136: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-15 17:11:55,136: INFO: 986331348: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=ce9551b9-7db8-4f56-858f-4496daea176a
To: c:\Users\HP\Desktop\Python\Vs_Python\Projects\Kidney_Disease_MLop_project\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:36<00:00, 1.58MB/s]

[2025-07-15 17:12:35,237: INFO: 986331348: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
